<a href="https://colab.research.google.com/github/rumman-adnan/Assignment-Sensors-Data/blob/main/EnsembleAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np

# Load train and test datasets
train_data = np.load('/content/drive/MyDrive/sensors data/simu_20000_0.1_90_140_train.npy')

test_data = np.load('/content/drive/MyDrive/sensors data/simu_10000_0.1_141_178_test.npy')

# For training data
X_train = train_data[:, :1000]
y_train_systolic = train_data[:, -2]
y_train_diastolic = train_data[:, -1]

# For test data
X_test = test_data[:, :1000]
y_test_systolic = test_data[:, -2]
y_test_diastolic = test_data[:, -1]

# Displaying shapes to verify the split
X_train.shape, y_train_systolic.shape, y_train_diastolic.shape, X_test.shape, y_test_systolic.shape, y_test_diastolic.shape


((20000, 1000), (20000,), (20000,), (10000, 1000), (10000,), (10000,))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

# Training models for systolic and diastolic pressures
model_systolic = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model_diastolic = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Fit the models
model_systolic.fit(X_train, y_train_systolic)
model_diastolic.fit(X_train, y_train_diastolic)

# Predicting on the test set
y_pred_systolic = model_systolic.predict(X_test)
y_pred_diastolic = model_diastolic.predict(X_test)

# Calculate the MAE for both systolic and diastolic predictions
mae_systolic = mean_absolute_error(y_test_systolic, y_pred_systolic)
mae_diastolic = mean_absolute_error(y_test_diastolic, y_pred_diastolic)

mae_systolic, mae_diastolic


(35.06195809453011, 10.110753746633229)

In [ ]:
# Load train and test datasets
train_data = np.load('/content/drive/MyDrive/sensors data/simu_20000_0.1_90_140_train.npy')

test_data = np.load('/content/drive/MyDrive/sensors data/simu_10000_0.1_141_178_test.npy')

# For training data
X_train = train_data[:, :1000]
y_train_systolic = train_data[:, -2]
y_train_diastolic = train_data[:, -1]

# For test data
X_test = test_data[:, :1000]
y_test_systolic = test_data[:, -2]
y_test_diastolic = test_data[:, -1]


In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf

# Reshape data for LSTM [samples, time steps, features]
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Combine Systolic and Diastolic BP for training
y_train = np.stack([y_train_systolic, y_train_diastolic], axis=1)

# Define LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(2)  # Two output neurons for S and D
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
y_pred_systolic = y_pred[:, 0]
y_pred_diastolic = y_pred[:, 1]
mae_systolic = np.mean(np.abs(y_pred_systolic - y_test_systolic))
mae_diastolic = np.mean(np.abs(y_pred_diastolic - y_test_diastolic))

print(f"Test MAE for Systolic BP: {mae_systolic}")
print(f"Test MAE for Diastolic BP: {mae_diastolic}")


Epoch 1/50
250/250 [==============================] - 141s 558ms/step - loss: nan - val_loss: nan
Epoch 2/50
 81/250 [========>.....................] - ETA: 1:14 - loss: nan

KeyboardInterrupt: ignored